In [ ]:
import cv2
import glob, os, sys
import mediapipe as mp
import numpy as np
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import multiprocessing 

In [ ]:
mp_holistic = mp.solutions.holistic

DATA_PATH = "dataset/ChicagoFSWildPlus/ChicagoFSWildPlus/"
OUT_PATH = "dataset/mediapipe_res_chicago_2/"

BG_COLOR = (192, 192, 192) # gray

BODY_POINTS = mp_holistic.PoseLandmark._member_names_
BODY_LIMBS = [(int(a), int(b)) for a, b in mp_holistic.POSE_CONNECTIONS]

HAND_POINTS = mp_holistic.HandLandmark._member_names_
HAND_LIMBS = [(int(a), int(b)) for a, b in mp_holistic.HAND_CONNECTIONS]

FACE_POINTS_NUM = lambda additional_points=0: additional_points + 468
FACE_POINTS = lambda additional_points=0: [str(i) for i in range(FACE_POINTS_NUM(additional_points))]
FACE_LIMBS = [(int(a), int(b)) for a, b in mp_holistic.FACEMESH_TESSELATION]


def component_points(component, width: int, height: int, num: int):
    if component is not None:
        lm = component.landmark
        return np.array([[p.x * width, p.y * height, p.z] for p in lm]), np.ones(num)

    return np.zeros((num, 3)), np.zeros(num)


def body_points(component, width: int, height: int, num: int):
    if component is not None:
        lm = component.landmark
        return np.array([[p.x * width, p.y * height, p.z] for p in lm]), np.array([p.visibility for p in lm])

    return np.zeros((num, 3)), np.zeros(num)

In [ ]:
def process_batch_folder(p1dir):
    for p2dir in os.listdir(DATA_PATH+p1dir):

        IMAGE_FILES = sorted(glob.glob(DATA_PATH+ p1dir + "/" + p2dir +"/*.jpg"))
                
        for idx, file in enumerate(IMAGE_FILES):
            #print("" + count + str(round(count/cpt, 3)) + "%")
            
            image = cv2.imread(file)
            image_height, image_width, _ = image.shape
            h, w, _ = image.shape

            # Convert the BGR image to RGB before processing.
            results = holistic.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            body_data, body_confidence = body_points(results.pose_landmarks, w, h, 33)
            face_data, face_confidence = component_points(results.face_landmarks, w, h,
                                                    FACE_POINTS_NUM(0))
            lh_data, lh_confidence = component_points(results.left_hand_landmarks, w, h, 21)
            rh_data, rh_confidence = component_points(results.right_hand_landmarks, w, h, 21)
            body_world_data, body_world_confidence = body_points(results.pose_world_landmarks, w, h, 33)
            
            data = np.concatenate([body_data, face_data, lh_data, rh_data, body_world_data])
            conf = np.concatenate([body_confidence, face_confidence, lh_confidence, rh_confidence, body_world_confidence])
        
            datas.append(data)
            confs.append(conf)
            
            numpy_res = np.concatenate((data,conf[:,None]),axis= 1)
            #save the results as .npy file
            
            import os
            path = OUT_PATH  + p1dir  + "/" + p2dir + "/" + file.split(p2dir)[1][1:-3] + "npy"
            os.makedirs(OUT_PATH + p1dir  + "/" + p2dir + "/" , exist_ok=True)
            np.save(path, numpy_res) 
            
with mp_holistic.Holistic(
    static_image_mode=False,
    model_complexity=2,
    enable_segmentation=False,
    refine_face_landmarks=False) as holistic:

  datas = []
  confs = []
  count = 0
  for p1dir in os.listdir(DATA_PATH):
    process_batch_folder(p1dir)

In [7]:
from time import sleep
from multiprocessing import Pool
def f(x):
    #sleep(x/10)
    print(x)
with Pool() as p:
    p.map(f, range(30))

12346570810

9


11





12131415
1716181920
2122


23





24
252627
2928





